In [1]:
import polars as pl
import bioframe as bf
from IPython.display import display
import os
os.chdir("/zata/zippy/ramirezc/splice-model-benchmark")

In [2]:
gtf = pl.read_parquet('/zata/zippy/ramirezc/static_files/gencode.v29.basic.annotation.gtf.parquet')
display(gtf)
example_intron_df = pl.read_csv('golden_standard/downloads/ENCBS020DQC/ENCFF835KGV.introns.tab', separator='\t')
display(example_intron_df)

seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
cat,cat,cat,i64,i64,f32,cat,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""chr1""","""HAVANA""","""gene""",11869,14409,null,"""+""",0,"""ENSG00000223972.5""","""transcribed_unprocessed_pseudo…","""DDX11L1""","""2""","""OTTHUMG00000000961.2""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""chr1""","""HAVANA""","""transcript""",11869,14409,null,"""+""",0,"""ENSG00000223972.5""","""transcribed_unprocessed_pseudo…","""DDX11L1""","""2""","""OTTHUMG00000000961.2""","""ENST00000456328.2""","""processed_transcript""","""DDX11L1-202""","""1""","""basic""","""OTTHUMT00000362751.1""","""""","""""","""""","""""",""""""
"""chr1""","""HAVANA""","""exon""",11869,12227,null,"""+""",0,"""ENSG00000223972.5""","""transcribed_unprocessed_pseudo…","""DDX11L1""","""2""","""OTTHUMG00000000961.2""","""ENST00000456328.2""","""processed_transcript""","""DDX11L1-202""","""1""","""basic""","""OTTHUMT00000362751.1""","""1""","""ENSE00002234944.1""","""""","""""",""""""
"""chr1""","""HAVANA""","""exon""",12613,12721,null,"""+""",0,"""ENSG00000223972.5""","""transcribed_unprocessed_pseudo…","""DDX11L1""","""2""","""OTTHUMG00000000961.2""","""ENST00000456328.2""","""processed_transcript""","""DDX11L1-202""","""1""","""basic""","""OTTHUMT00000362751.1""","""2""","""ENSE00003582793.1""","""""","""""",""""""
"""chr1""","""HAVANA""","""exon""",13221,14409,null,"""+""",0,"""ENSG00000223972.5""","""transcribed_unprocessed_pseudo…","""DDX11L1""","""2""","""OTTHUMG00000000961.2""","""ENST00000456328.2""","""processed_transcript""","""DDX11L1-202""","""1""","""basic""","""OTTHUMT00000362751.1""","""3""","""ENSE00002312635.1""","""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""chrM""","""ENSEMBL""","""transcript""",15888,15953,null,"""+""",0,"""ENSG00000210195.2""","""Mt_tRNA""","""MT-TT""","""3""","""""","""ENST00000387460.2""","""Mt_tRNA""","""MT-TT-201""","""NA""","""basic""","""""","""""","""""","""""","""""",""""""
"""chrM""","""ENSEMBL""","""exon""",15888,15953,null,"""+""",0,"""ENSG00000210195.2""","""Mt_tRNA""","""MT-TT""","""3""","""""","""ENST00000387460.2""","""Mt_tRNA""","""MT-TT-201""","""NA""","""basic""","""""","""1""","""ENSE00001544475.2""","""""","""""",""""""
"""chrM""","""ENSEMBL""","""gene""",15956,16023,null,"""-""",0,"""ENSG00000210196.2""","""Mt_tRNA""","""MT-TP""","""3""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


chrom,start,end,strand,reads,reads_per_million
str,i64,i64,str,i64,f64
"""chr1""",827775,829002,"""+""",4,2.365377
"""chr1""",829104,841199,"""+""",6,3.548066
"""chr1""",827853,829002,"""+""",1,0.591344
"""chr1""",960800,961628,"""+""",5,2.956722
"""chr1""",961750,961825,"""+""",16,9.46151
…,…,…,…,…,…
"""chrX""",155514265,155524455,"""-""",11,6.504788
"""chrX""",155524632,155612791,"""-""",2,1.182689
"""chrX""",155524632,155545095,"""-""",12,7.096132


In [3]:
slim_gtf = gtf.select(['seqname', 'start', 'end', 'strand', 'feature', 'gene_id', 'gene_name', 'transcript_id', 'gene_type']).rename({'seqname': 'chrom'})
filter_gtf = slim_gtf.filter(pl.col('feature') == 'gene', pl.col('gene_type') == 'protein_coding')
display(filter_gtf)

chrom,start,end,strand,feature,gene_id,gene_name,transcript_id,gene_type
cat,i64,i64,cat,cat,str,str,str,str
"""chr1""",65419,71585,"""+""","""gene""","""ENSG00000186092.6""","""OR4F5""","""""","""protein_coding"""
"""chr1""",450703,451697,"""-""","""gene""","""ENSG00000284733.1""","""OR4F29""","""""","""protein_coding"""
"""chr1""",685679,686673,"""-""","""gene""","""ENSG00000284662.1""","""OR4F16""","""""","""protein_coding"""
"""chr1""",923928,944581,"""+""","""gene""","""ENSG00000187634.11""","""SAMD11""","""""","""protein_coding"""
"""chr1""",944204,959309,"""-""","""gene""","""ENSG00000188976.10""","""NOC2L""","""""","""protein_coding"""
…,…,…,…,…,…,…,…,…
"""chrM""",10470,10766,"""+""","""gene""","""ENSG00000212907.2""","""MT-ND4L""","""""","""protein_coding"""
"""chrM""",10760,12137,"""+""","""gene""","""ENSG00000198886.2""","""MT-ND4""","""""","""protein_coding"""
"""chrM""",12337,14148,"""+""","""gene""","""ENSG00000198786.2""","""MT-ND5""","""""","""protein_coding"""


In [5]:
overlaps = bf.overlap(
    filter_gtf.to_pandas(),
    example_intron_df.to_pandas(),
    how='inner',
    suffixes=('_gene', '_intron')
)
display(overlaps)

contained = overlaps[
    (overlaps['start_intron'] >= overlaps['start_gene']) & 
    (overlaps['end_intron'] <= overlaps['end_gene'])
]
contained_pl = pl.from_pandas(contained)
unique_introns = contained_pl.unique(
    subset=['chrom_intron', 'start_intron', 'end_intron', 'strand_intron', 'reads_intron']
).rename(
    {
        'chrom_intron': 'chrom',
        'start_intron': 'start',
        'end_intron': 'end',
        'strand_intron': 'strand',
        'reads_intron': 'reads',
        'gene_name_gene': 'gene_name',
    }
).select('chrom', 'start', 'end', 'strand', 'reads', 'gene_name').sort('chrom', 'start')

display(unique_introns)

,chrom_gene,start_gene,end_gene,strand_gene,feature_gene,gene_id_gene,gene_name_gene,transcript_id_gene,gene_type_gene,chrom_intron,start_intron,end_intron,strand_intron,reads_intron,reads_per_million_intron
0,chr7,192969,260745,+,gene,ENSG00000177706.8,FAM20C,,protein_coding,chr7,193804,195553,+,13,7.687477
1,chr7,192969,260745,+,gene,ENSG00000177706.8,FAM20C,,protein_coding,chr7,193804,208897,+,1,0.591344
2,chr7,192969,260745,+,gene,ENSG00000177706.8,FAM20C,,protein_coding,chr7,195732,208897,+,12,7.096132
3,chr7,192969,260745,+,gene,ENSG00000177706.8,FAM20C,,protein_coding,chr7,208976,246414,+,13,7.687477
4,chr7,192969,260745,+,gene,ENSG00000177706.8,FAM20C,,protein_coding,chr7,246507,248314,+,13,7.687477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236492,chr18,49488453,49492523,-,gene,ENSG00000265681.7,RPL17,,protein_coding,chr18,49483771,49489358,-,2,1.182689
236493,chr18,49795793,49837623,-,gene,ENSG00000266997.2,AC090227.1,,protein_coding,chr18,49787361,49813816,-,1,0.591344
236494,chr18,49795793,49837623,-,gene,ENSG00000266997.2,AC090227.1,,protein_coding,chr18,49794427,49797465,-,1,0.591344
236495,chr18,57646426,57803315,-,gene,ENSG00000081923.13,ATP8B1,,protein_coding,chr18,57601783,57655193,-,1,0.591344


chrom,start,end,strand,reads,gene_name
str,i64,i64,str,i64,str
"""chr1""",944550,945041,"""-""",1,"""NOC2L"""
"""chr1""",944750,945056,"""-""",1,"""NOC2L"""
"""chr1""",944800,945056,"""-""",284,"""NOC2L"""
"""chr1""",944800,945041,"""-""",141,"""NOC2L"""
"""chr1""",944819,945041,"""-""",1,"""NOC2L"""
…,…,…,…,…,…
"""chrX""",155898249,155919812,"""+""",1,"""VAMP7"""
"""chrX""",155900587,155939700,"""+""",2,"""VAMP7"""
"""chrX""",155900587,155919812,"""+""",83,"""VAMP7"""


Started with 225_467 introns

~~After filtering, there are 230_405 (buh)~~

Had to filter for unique introns, because the gtf file has some overlapping genes don't map to one entry in the gtf file 1-to-1 I guess.

In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import numpy as np

reads = np.empty(0, dtype=np.float64)
for file_path in input.all_introns_files:
    df = pl.read_csv(file_path, separator='\t')
    reads = np.append(reads, df['reads'].to_numpy())

plt.style.use('cowplot')
fig, axes = plt.subplots(figsize=(5, 5))
fig.suptitle('Intron read distribution')
axes.hist(reads, bins=1000, alpha=0.5)
axes.set_xlabel('Number of Reads')
axes.set_ylabel('Count')
ymin, ymax = axes.get_ylim()
padding = 0.01 * ymax
new_bottom = ymin - padding
axes.set_ylim(bottom=new_bottom)

axes.margins(y=0.01)
plt.savefig(output.plot)

shape: (212_986, 5)
┌───────┬───────────┬───────────┬────────┬───────┐
│ chrom ┆ start     ┆ end       ┆ strand ┆ reads │
│ ---   ┆ ---       ┆ ---       ┆ ---    ┆ ---   │
│ str   ┆ i64       ┆ i64       ┆ str    ┆ i64   │
╞═══════╪═══════════╪═══════════╪════════╪═══════╡
│ chr1  ┆ 944550    ┆ 945041    ┆ -      ┆ 1     │
│ chr1  ┆ 944750    ┆ 945056    ┆ -      ┆ 1     │
│ chr1  ┆ 944800    ┆ 945041    ┆ -      ┆ 141   │
│ chr1  ┆ 944800    ┆ 945056    ┆ -      ┆ 284   │
│ chr1  ┆ 944819    ┆ 945041    ┆ -      ┆ 1     │
│ …     ┆ …         ┆ …         ┆ …      ┆ …     │
│ chrX  ┆ 155898249 ┆ 155900496 ┆ +      ┆ 84    │
│ chrX  ┆ 155900587 ┆ 155939700 ┆ +      ┆ 2     │
│ chrX  ┆ 155900587 ┆ 155919812 ┆ +      ┆ 83    │
│ chrX  ┆ 155919880 ┆ 155939700 ┆ +      ┆ 84    │
│ chrX  ┆ 155939793 ┆ 155941882 ┆ +      ┆ 86    │
└───────┴───────────┴───────────┴────────┴───────┘
